In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Define the Bayesian neural network
class BayesianNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BayesianNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the function to fit (sine function)
def sine_function(x):
    return np.sin(x)

# Generate data
torch.manual_seed(42)
np.random.seed(42)
X_train = np.random.uniform(-5, 5, 1000).reshape(-1, 1).astype(np.float32)
y_train = sine_function(X_train)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

# Define the Bayesian neural network model
input_size = 1
hidden_size = 100
output_size = 1
model = BayesianNN(input_size, hidden_size, output_size)

# Define the loss function
def gaussian_likelihood(y_pred, y_actual, sigma):
    exponent = -0.5 * (1 / sigma**2) * (y_pred - y_actual)**2
    normalization = torch.sqrt(torch.tensor(2 * np.pi) * sigma**2)
    return torch.exp(exponent) / normalization

# Define the prior distribution over weights
prior_mu = 0
prior_sigma = 1

# Define the variance of the likelihood function
likelihood_sigma = 0.1

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 2000
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    # Forward pass
    outputs = model(X_train)
    # Calculate likelihood
    likelihood = gaussian_likelihood(outputs, y_train, likelihood_sigma)
    # Calculate prior
    prior = torch.empty(0)
    for param in model.parameters():
        prior = torch.cat((prior, torch.randn_like(param).mul(prior_sigma).add(prior_mu)))
    prior = gaussian_likelihood(outputs, prior, prior_sigma)
    # Calculate posterior
    posterior = likelihood * prior
    # Calculate loss
    loss = -posterior.log().sum()
    # Backward pass
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Prediction
model.eval()
with torch.no_grad():
    outputs = torch.cat([model(X_train) for _ in range(100)], dim=1)
    mean_prediction = outputs.mean(dim=1).numpy()
    std_prediction = outputs.std(dim=1).numpy()

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(X_train.numpy(), y_train.numpy(), color='blue', label='Original data')
plt.plot(X_train.numpy(), mean_prediction, color='red', label='Prediction')
plt.fill_between(X_train.numpy().flatten(), mean_prediction - 2 * std_prediction, mean_prediction + 2 * std_prediction, color='gray', alpha=0.2, label='Uncertainty')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Sine Function Fitting with Bayesian Neural Network')
plt.legend()
plt.grid(True)
plt.show()

RuntimeError: Tensors must have same number of dimensions: got 2 and 1